# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
display (data.head())

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df=data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
display (df.head())

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pvt=df.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
display (pvt.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
cust_similar=squareform(pdist(pvt.T, 'euclidean'))
#display (cust_similar)

In [7]:
distancias=pd.DataFrame(1/(1+cust_similar), index=pvt.columns, columns=pvt.columns)
display (distancias.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
similaridad=distancias[33].sort_values(ascending=False)[1:]  # cliente ID=33
top5=similaridad.head().index.to_list()
print (top5)

[264, 3535, 3317, 2503, 3305]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
selection=df[df['CustomerID'].isin(top5)]
display (selection.head())

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
quantity=selection.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
display (quantity.head())

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
no_buy=pvt[pvt[33]==0]
merged=no_buy.merge(quantity, left_on='ProductName', right_on='ProductName').sort_values('Quantity', ascending=False)
display (merged[[33, 'Quantity']].head())

,33,Quantity
ProductName,,
Wine - Blue Nun Qualitatswein,0.0,3
Soup - Campbells Bean Medley,0.0,3
Wine - Ej Gallo Sierra Valley,0.0,3
Butter - Unsalted,0.0,3
Yoghurt Tubes,0.0,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
recomendaciones={}
cust=list(data['CustomerID'].unique())

for c in cust:
    similaridad=distancias[c].sort_values(ascending=False)[1:]  # cliente ID=33
    top5=similaridad.head().index.to_list()
    selection=df.loc[df['CustomerID'].isin(top5)]
    similar_df=selection.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
    merged=similar_df.merge(pvt, left_on='ProductName', right_on='ProductName')
    no_buy=merged[merged[c]==0]
    quantity=no_buy.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()
    recomendaciones.update({c: quantity})

In [13]:
# print (recomendaciones)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
final_df=pd.DataFrame(recomendaciones)
display (final_df.head())

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Squid U5 - Thailand,Cocktail Napkin Blue,Grouper - Fresh,Banana Turning,Lettuce - Frisee,"Cheese - Boursin, Garlic / Herbs",Scallops - 10/20,Bandage - Flexible Neon,Ketchup - Tomato,Cheese - Mozzarella,...,Halibut - Steaks,"Doilies - 5, Paper",Milk - 2%,Rum - Mount Gay Eclipes,"Rosemary - Primerba, Paste","Pasta - Penne, Rigate, Dry",Beef - Striploin Aa,Wine - White Cab Sauv.on,Cup - Translucent 7 Oz Clear,Wine - Chardonnay South
1,Pasta - Orecchiette,Chicken - Soup Base,Sherry - Dry,Soupcontfoam16oz 116con,Vanilla Beans,Garlic - Peeled,Skirt - 29 Foot,Nantucket - Pomegranate Pear,Cod - Black Whole Fillet,Cheese - Parmesan Grated,...,Beef - Striploin Aa,Soup V8 Roasted Red Pepper,Bay Leaf,Muffin Mix - Blueberry,"Bread - Roll, Soft White Round",Beets - Mini Golden,"Thyme - Lemon, Fresh",Olives - Kalamata,Yogurt - French Vanilla,Bar - Granola Trail Mix Fruit Nut
2,Ezy Change Mophandle,Cheese - Taleggio D.o.p.,Halibut - Steaks,"Appetizer - Mini Egg Roll, Shrimp",Beef - Inside Round,Cheese - Bocconcini,"Spoon - Soup, Plastic",Scallops - 10/20,Rabbit - Whole,Potatoes - Idaho 100 Count,...,Mussels - Frozen,Phyllo Dough,Cake - Cake Sheet Macaroon,Assorted Desserts,Flour - Whole Wheat,"Garlic - Primerba, Paste",Bread Foccacia Whole,Pasta - Angel Hair,Flour - Teff,Butter - Unsalted
3,Dc Hikiage Hira Huba,Grenadine,"Oregano - Dry, Rubbed",Wine - Chardonnay South,Sausage - Liver,Turkey - Oven Roast Breast,Sobe - Tropical Energy,Bread - French Baquette,"Veal - Brisket, Provimi,bnls","Lemonade - Natural, 591 Ml",...,Wine - Charddonnay Errazuriz,Cocoa Butter,Muffin Chocolate Individual Wrap,Cheese - Mix,Chips Potato Salt Vinegar 43g,Juice - Orange,Tea - Earl Grey,Meldea Green Tea Liquor,Beef - Ground Medium,Wine - Ej Gallo Sierra Valley
4,Wine - Prosecco Valdobiaddene,Guinea Fowl,Beef - Inside Round,Tuna - Salad Premix,Pop Shoppe Cream Soda,Cocoa Butter,Pastry - Choclate Baked,Thermometer Digital,"Thyme - Lemon, Fresh",V8 - Berry Blend,...,Beef - Top Sirloin,"Oil - Shortening,liqud, Fry","Appetizer - Mini Egg Roll, Shrimp",Scallops 60/80 Iqf,Fenngreek Seed,Chinese Foods - Chicken,"Fish - Scallops, Cold Smoked",V8 - Berry Blend,Oil - Shortening - All - Purpose,Sardines


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [15]:
cust_similar2=squareform(pdist(pvt.T, 'cityblock'))
distancias2=pd.DataFrame(1/(1+cust_similar2), index=pvt.columns, columns=pvt.columns)

recomendaciones2={}

for c in cust:
    similaridad=distancias2[c].sort_values(ascending=False)[1:]  # cliente ID=33
    top5=similaridad.head().index.to_list()
    selection=df.loc[df['CustomerID'].isin(top5)]
    similar_df=selection.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
    merged=similar_df.merge(pvt, left_on='ProductName', right_on='ProductName')
    no_buy=merged[merged[c]==0]
    quantity=no_buy.sort_values('Quantity',ascending=False)['ProductName'][0:5].tolist()
    recomendaciones2.update({c: quantity})

In [16]:
#print (recomendaciones2)

In [17]:
final_df2=pd.DataFrame(recomendaciones2)
display (final_df2.head())

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Beans - Wax,Table Cloth 81x81 White,Butter - Unsalted,"Wine - Magnotta, Merlot Sr Vqa",Wine - Vidal Icewine Magnotta,"Pepper - Black, Whole",Sobe - Tropical Energy,Pork - Kidney,Butter - Unsalted,Squid - Tubes / Tenticles 10/20,...,Pork - Kidney,Soup V8 Roasted Red Pepper,Cheese - Parmesan Grated,Soup V8 Roasted Red Pepper,Wine - Chardonnay South,Sardines,Pork - Kidney,Pork - Kidney,Juice - Apple Cider,Pork - Kidney
1,Knife Plastic - White,Wine - Vidal Icewine Magnotta,Sardines,"Pepper - White, Ground",Table Cloth 81x81 White,Beans - Wax,Skirt - 29 Foot,Lime Cordial - Roses,Muffin - Carrot Individual Wrap,Bar - Granola Trail Mix Fruit Nut,...,Muffin - Carrot Individual Wrap,Quiche Assorted,Bar - Granola Trail Mix Fruit Nut,Beans - Wax,Butter - Unsalted,Knife Plastic - White,Knife Plastic - White,Wine - Chardonnay South,Juice - Lime,Bar - Granola Trail Mix Fruit Nut
2,Ice Cream Bar - Oreo Cone,Beets - Mini Golden,Knife Plastic - White,Wine - Chardonnay South,"Salsify, Organic",Bar - Granola Trail Mix Fruit Nut,Smirnoff Green Apple Twist,"Cheese - Boursin, Garlic / Herbs","Pepper - Black, Whole",Wine - Toasted Head,...,Beets - Mini Golden,Crush - Cream Soda,Butter - Unsalted,Beans - Wax,Beans - Wax,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Butter - Unsalted,Tomato - Tricolor Cherry,Sardines
3,Bread - Rye,Soup - Campbells Tomato Ravioli,Wine - Chardonnay South,"Cheese - Boursin, Garlic / Herbs",Knife Plastic - White,Muffin - Carrot Individual Wrap,Butter - Unsalted,Bread - English Muffin,Cod - Black Whole Fillet,Durian Fruit,...,Lime Cordial - Roses,V8 - Berry Blend,"Mushrooms - Black, Dried","Oil - Shortening,liqud, Fry","Pepper - White, Ground",Fond - Neutral,Cheese - Bocconcini,Sobe - Tropical Energy,"Pepper - Black, Whole",Muffin - Carrot Individual Wrap
4,Bay Leaf,Muffin - Carrot Individual Wrap,Bar - Granola Trail Mix Fruit Nut,Ice Cream Bar - Hageen Daz To,"Veal - Inside, Choice",Pomello,"Pepper - Black, Whole",Cheese - Parmesan Cubes,Knife Plastic - White,"Salsify, Organic",...,Pasta - Angel Hair,"Veal - Brisket, Provimi,bnls",Juice - Apple Cider,"Wine - Magnotta, Merlot Sr Vqa","Cheese - Boursin, Garlic / Herbs",Crush - Cream Soda,Vanilla Beans,Knife Plastic - White,Soup - Campbells Tomato Ravioli,Lime Cordial - Roses
